# Condo Ownership vs Stock Index Investment

In [2]:
import numpy as np

## Yearly Expenditures for Condo Ownership (3494 hotel-de-ville)

(Property Tax for [Plateau Mont-Royale](http://ville.montreal.qc.ca/pls/portal/docs/PAGE/SERVICE_FIN_EN/MEDIA/DOCUMENTS/2019_PLATEAU_ANG.PDF)) * (Most recent property evaluation for 3494 hotel-de-ville)


In [3]:
plateau_tax_rate = 0.006519 + 0.000025 + 0.001083 + 0.000036 + 0.000591 + 0.000315
municipal_evaluation = 359000
PROPERTY_TAX = plateau_tax_rate * municipal_evaluation

In [4]:
CONDO_FEES = 2400
INSURANCE = 1000
MAINTENANCE = 1500

In [5]:
ANNUAL_EXPENSE = sum([
    CONDO_FEES,
    INSURANCE,
    MAINTENANCE,
    PROPERTY_TAX
])
ANNUAL_EXPENSE

7976.270999999999

## One-time Expenses for Condo Ownership

In [6]:
welcome_tax = 4500
notary = 1500
title_insurance = 400
inspection = 1000

INITIAL_EXPENSE = sum([welcome_tax, notary, title_insurance, inspection])
INITIAL_EXPENSE

7400

## Mortgage Analysis

### CMHC Insurance

Interest rates [listed here](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/mortgage-loan-insurance-homeownership-programs/cmhc-mortgage-loan-insurance-cost). Mortgage insurance is covered by the lender for loan to value of under 80% (i.e., if your downpayment is at least 20%, this section is of no concern to you).

In [7]:
def mortgage_insurance_rate(p, downpayment):
    ltv = (p - downpayment) / p
    assert ltv > 0
    assert ltv <= 1
    
    if ltv >= 0.95:
        return 0.04
    elif ltv >= 0.90:
        return 0.031
    elif ltv >= 0.85:
        return 0.028
    elif ltv >= 0.80:
        return 0.024
    else:
        return 0

In [8]:
def mortgage(price, downpayment, interest, periods):
    initial_principal = price + mortgage_insurance_rate(price, downpayment) * price - downpayment
    ppmt = -np.ppmt(interest, np.arange(periods) + 1, periods, initial_principal)
    ipmt = -np.ipmt(interest, np.arange(periods) + 1, periods, initial_principal)
    return ppmt, ipmt

def print_mortgage(ppmt, ipmt):
    assert(len(ppmt) == len(ipmt))
    print(f"with a monthly payment of {(ppmt[0] + ipmt[0]) / 12}...")
    fmt = '{0:2d} {4:8.2f} {1:8.2f} {2:8.2f} {3:8.2f}'
    p = initial_principal
    per = np.arange(len(ppmt))
    for payment in per:
        index = payment - 1
        p = p - ppmt[index]
        print(fmt.format(payment, ppmt[index], ipmt[index], p, ppmt[index] + ipmt[index]))

### Gross Debt Service (GDS)

A GDS ratio is the percentage of your income needed to pay all of your monthly housing costs, including principal, interest, taxes, and heat (PITH). You’ll also need to include 50 per cent of your condo fees, if applicable.

### Total Debt Service Ratio

Includes other debt obligations (credit cards, lines of credit, car loans, etc.)

You can include up to 50% of your rental income, and in this case, remove heating expenses from the calculation.

Canadian gov [reference](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/calculating-gds-tds)

### Capital Tax

In [10]:
def capital_gains_tax(profit, income):
    federal, quebec = 0.15, 0.15
    if income > 43055:
        quebec = 0.20
    if income > 47630:
        federal = 0.205
    if income > 86105:
        quebec = 0.24
    if income > 95259:
        federal = 0.26
    if income > 104765:
        quebec = 0.2575
    if income > 147740:
        federal = 0.29
    if income > 210371:
        federal = 0.33
    
    # by law, only half of capital gains are taxed
    taxable_profit = profit / 2
    return (federal + quebec) * taxable_profit

### Comparative Yearly Return

Condominium appreciation 
[Plateau Mont Royale centris](https://www.centris.ca/en/tools/real-estate-statistics/montreal-island/le-plateau-mont-royal-montreal) -- 8% last year

[Montreal shupilov.com](https://news.shupilov.com/blog/average-real-estate-prices-and-appreciation-rates-in-montreal/) -- 3% per year

[fciq.ca](https://www.fciq.ca/pdf/mot_economiste/me_052014_an.pdf)

44% monthly cost salary as imposed by the CMHC insurance

In [11]:
def cumulative(np_array):
    return np.matmul(np_array, np.triu(np.ones(len(np_array) * len(np_array)).reshape((len(np_array), len(np_array)))))

def future_values(rate, nper, pmt, pv):
    return np.array([np.fv(rate, n, pmt, pv) for n in range(nper)])

def rental_income(start, incr, years):
    return np.array([12 * (starting_rent + i*constant_rent_increase) for i in range(nper)])

def net_operating_x(*np_arrays):
    return np.array([max(0, x) for x in sum(np_arrays)])

def realestate_value(appreciation, per, start):
    return future_values(appreciation, per, 0, -start)

def stocks(growth, capital, nper):
    average_pmt = np.average(capital[1:10])
    gains = future_values(growth, nper, -average_pmt, -capital[0])
    profit = gains - cumulative(capital)
    return gains - capital_gains_tax(profit, FUTURE_INCOME_AT_SELLTIME)

def realestate(price, appreciation, nper, pmt, ipmt, closing_cost, noi, nol):
    sell_price = realestate_value(appreciation, nper, price)
    gains = sell_price + cumulative(noi)
    losses = closing_cost * sell_price + cumulative(nol) + (sum(pmt) - cumulative(pmt))
    profit = gains - losses
    return gains - losses - capital_gains_tax(profit, FUTURE_INCOME_AT_SELLTIME)

In [36]:
interest = 0.03
nper = 25
per = np.arange(nper) + 1
price = 375000
downpayment = 20000
condo_appreciation = 0.05
starting_rent = 1910
constant_rent_increase = 40

# To sell it
closing_cost = 0.04
FUTURE_INCOME_AT_SELLTIME = 100000

initial_capital = downpayment + INITIAL_EXPENSE
pmt, ipmt = mortgage(price, downpayment, interest, 25)

income = rental_income(starting_rent, constant_rent_increase, nper)
losses = np.array([INITIAL_EXPENSE + ANNUAL_EXPENSE + downpayment] + [ANNUAL_EXPENSE] * (nper - 1)) + pmt + ipmt
net = income - losses
noi = np.array([max(0, x) for x in net])
nol = -np.array([min(0, x) for x in net])

r = realestate(price, condo_appreciation, nper, pmt, ipmt, closing_cost, noi, nol)
s = stocks(0.07, nol, nper)

In [40]:
def display_comparison(s, r):
    print("year | stocks | realestate | ratio | principal | interest | noi |  ")
    for i in range(len(s)):
        print(f"{i:4} | {s[i]:6.0f} | {r[i]:10.0f} | {r[i]/s[i]:5.2f} | {pmt[i]/12:9.0f} | {ipmt[i]/12:8.0f} | {net[i]:4.0f}")
display_comparison(s, r)

year | stocks | realestate | ratio | principal | interest | noi |  
   0 |  33511 |     -22560 | -0.67 |       838 |      917 | -33511
   1 |  39461 |      -5515 | -0.14 |       863 |      891 | -5631
   2 |  45609 |      12798 |  0.28 |       889 |      866 | -5151
   3 |  51977 |      32420 |  0.62 |       916 |      839 | -4671
   4 |  58589 |      53393 |  0.91 |       943 |      811 | -4191
   5 |  65471 |      75762 |  1.16 |       971 |      783 | -3711
   6 |  72650 |      99574 |  1.37 |      1001 |      754 | -3231
   7 |  80155 |     124878 |  1.56 |      1031 |      724 | -2751
   8 |  88016 |     151725 |  1.72 |      1062 |      693 | -2271
   9 |  96269 |     180168 |  1.87 |      1093 |      661 | -1791
  10 | 104947 |     210263 |  2.00 |      1126 |      628 | -1311
  11 | 114091 |     242070 |  2.12 |      1160 |      595 | -831
  12 | 123739 |     275649 |  2.23 |      1195 |      560 | -351
  13 | 133970 |     311065 |  2.32 |      1231 |      524 |  129
  14 | 144

In [17]:
#calculate roi

roi_realestate = (realestate - cumulative_capital) / cumulative_capital
roi_realestate

NameError: name 'cumulative_capital' is not defined

In [ ]:
first_year = rental_income * 12 - annual_expense - initial_expense + ppmt[0] - ipmt[0]
def net(year):
    if year == 1:
        return first_year
    return net(year - 1) + rental_income * 12 - annual_expense + ppmt[year] - ipmt[year]

In [ ]:
np.fv(0.07, per, -80000, -50000)

In [ ]:
realestate / stocks

In [ ]:
realestate - stocks